# Quantum Tomography: Machine Learning (SVR) vs Maximum Likelihood Estimation (MLE)

## Project Overview
The goal of this notebook is to benchmark Machine Learning algorithms—specifically **Support Vector Regression (SVR)**—against standard **Maximum Likelihood Estimation (MLE)** for the task of single-qubit quantum tomography.

## Problem Statement
Quantum tomography consists of reconstructing the quantum state (density matrix $\rho$) from a set of measurements. 
- **MLE** is the standard statistical approach but requires computationally expensive optimization for every new state.
- **ML** approaches propose to "learn" the inversion map offline. Once trained, the reconstruction is nearly instantaneous.

## Methodology & Parameters
We will explore the performance trade-offs along three dimensions:
1.  **Dataset Properties:** Variations in dataset size (`n_states`), measurement noise (`n_shots`), and physical decoherence (`decoherence_level`).
2.  **Algorithm (SVR):** Testing different Kernels (RBF, Polynomial, Sigmoid) via `MultiOutputRegressor`.
3.  **Hyperparameter Optimization:** Using Grid Search to find optimal $C$, $\gamma$, and $\epsilon$ parameters.

## Comparison Metrics
1.  **Fidelity:** Physical closeness between the reconstructed state and the real state (1.0 is perfect).
2.  **MSE (Mean Squared Error):** Euclidean distance in the Bloch sphere.
3.  **Computational Cost:** Wall-clock time for reconstruction.

In [ ]:
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

# Attempt to import the provided local module
# Assuming the file 'saint_dtSet.py' is in a folder named 'gemi' or in the root path.
try:
    from dataset_build.saint_dtSet import generate_qubit_tomography_dataset_base, perform_mle_tomography
except ImportError:
    # Fallback if the file is in the same directory
    try:
        from saint_dtSet import generate_qubit_tomography_dataset_base, perform_mle_tomography
    except ImportError:
        raise ImportError("Please ensure 'saint_dtSet.py' is in the python path.")

# ------------------------------------------------------------------------------
# METRICS DEFINITION
# ------------------------------------------------------------------------------

def bloch_to_density_matrix(x, y, z):
    """
    Converts Bloch vector components to a 2x2 density matrix.
    rho = 1/2 * (I + x*sigma_x + y*sigma_y + z*sigma_z)
    """
    sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
    sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)
    identity = np.eye(2, dtype=complex)
    
    rho = 0.5 * (identity + x*sigma_x + y*sigma_y + z*sigma_z)
    return rho

def calculate_fidelity(row, pred_cols=('X_pred', 'Y_pred', 'Z_pred'), real_cols=('X_real', 'Y_real', 'Z_real')):
    """
    Calculates the quantum fidelity between the Real state and the Predicted state.
    
    Formula for single qubit using Bloch vectors r_real and r_pred:
    F(rho, sigma) = 0.5 * (1 + r_real.r_pred + sqrt((1 - |r_real|^2)(1 - |r_pred|^2)))
    
    Note: This analytic formula is faster than matrix sqrt operations.
    """
    # Extract vectors
    r_real = np.array([row[c] for c in real_cols])
    r_pred = np.array([row[c] for c in pred_cols])
    
    # Norms squared
    norm2_real = np.dot(r_real, r_real)
    norm2_pred = np.dot(r_pred, r_pred)
    
    # Dot product
    dot_prod = np.dot(r_real, r_pred)
    
    # Safety clip for sqrt (numerical stability)
    val_real = max(0.0, 1.0 - norm2_real)
    val_pred = max(0.0, 1.0 - norm2_pred)
    
    fidelity = 0.5 * (1.0 + dot_prod + np.sqrt(val_real * val_pred))
    
    # Clip fidelity to [0, 1] range to handle float discrepancies
    return min(max(fidelity, 0.0), 1.0)

def evaluate_performance(df_results, title="Evaluation"):
    """
    Computes global metrics (Average Fidelity, MSE) and prints a summary.
    Expects columns: X_real, Y_real, Z_real AND X_pred, Y_pred, Z_pred.
    """
    print(f"--- {title} ---")
    
    # 1. MSE (Mean Squared Error) on Bloch components
    y_true = df_results[['X_real', 'Y_real', 'Z_real']].values
    y_pred = df_results[['X_pred', 'Y_pred', 'Z_pred']].values
    mse = mean_squared_error(y_true, y_pred)
    
    # 2. Fidelity
    # Apply row-wise calculation
    fidelities = df_results.apply(calculate_fidelity, axis=1)
    avg_fidelity = fidelities.mean()
    std_fidelity = fidelities.std()
    
    print(f"MSE (Bloch Vector): {mse:.6f}")
    print(f"Average Fidelity:   {avg_fidelity:.6f} (+/- {std_fidelity:.6f})")
    
    return mse, avg_fidelity

print("Libraries loaded and Metrics defined.")